
##ColabPaint: Inpainting loops using RFDesign

Easy to use protein looping / inpainting using [RFDesign](https://www.science.org/doi/abs/10.1126/science.abn2100?af=R). 

In [1]:
#@title Input protein "contigs" and loop lengths, then hit `Runtime` -> `Run all`
import os
from google.colab import files
import hashlib
import re

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

jobname = 'test' #@param {type:"string"}
# remove whitespaces
basejobname = "".join(jobname.split())
basejobname = re.sub(r'\W+', '', basejobname)
jobname = add_hash(basejobname, basejobname)

contigs = 'A1-27,A33-60,A67-93,A101-126' #@param {type:"string"}
loop_lengths = '2-9,2-9,2-9' #@param {type:"string"}
# contigs='A1-45,3-9,A55-75,4-8,A84-126'

os.chdir('/content/')

contigs = contigs.replace(' ','').split(',')
loop_ranges = loop_lengths.replace(' ','').split(',')

#@markdown You will be prompted to upload a PDB file to continue.
custom_template_path = f"{jobname}_template/"
if not os.path.exists(custom_template_path):
  os.mkdir(custom_template_path)
uploaded = files.upload()
for fn in uploaded.keys():
  os.rename(fn, f"{jobname}_template/{fn}")
pdbname = fn.split('.')[0]



Saving able.pdb to able.pdb


In [2]:
#@title Make job files
def make_job_file(contigs, loop_sizes, outdir):
  if outdir[-1] != '/':
    outdir += '/'
  len_contigs = len(contigs)

  contig_str = ''
  for i in range(len_contigs - 1):
    contig_str += contigs[i]
    contig_str += ','
    contig_str += loop_sizes[i]
    contig_str += ','
  contig_str += contigs[-1]
  outname_suffix = '_'.join(contig_str.split(','))
  outname = outdir + pdbname + '_' + outname_suffix

  with open(f'{outname}.sh', 'w') as jobfile:
    jobfile.write('#!/bin/bash \n')
    pdbpath = custom_template_path + pdbname
    jobfile.write(f"pdb='{pdbpath}.pdb' \n")
    jobfile.write(f"contigs='{contig_str}' \n")
    jobfile.write(f"out='{outname}' \n")
    jobfile.write('num_designs=1 \n')
    jobfile.write('tmpl_conf=0.75 \n')
    script = "script='/content/RFDesign/inpainting/inpaint.py' \n"
    jobfile.write(script)
    jobfile.write('mkdir -p `dirname $out` \n')
    jobfile.write('python $script --pdb $pdb --contigs $contigs --dump_trb --dump_pdb --n_cycle 20 --out $out --num_designs $num_designs \n')

local_loop_path = custom_template_path + 'local_loops/'
if not os.path.exists(local_loop_path):
  os.mkdir(local_loop_path)

for i in range(len(loop_ranges)):
  local_contigs = contigs[i:i+2]
  loop_start, loop_stop = loop_ranges[i].split('-')
  loop_start = int(loop_start)
  loop_stop = int(loop_stop)
  for j in range(loop_start, loop_stop + 1):
    make_job_file(local_contigs, [str(j)], local_loop_path)


In [3]:
#@title Make minimal environment file

with open('env.yml', 'w') as envfile:
  envfile.write(
      """
  name: SE3-nvidia \n
  channels: \n
    - rusty1s \n
    - pytorch \n
    - dglteam \n
    - nvidia \n
    - conda-forge \n
    - defaults \n
  dependencies: \n
    - cudatoolkit=11.1.74=h6bb024c_0 \n
    - numpy=1.20.3=py39hf144106_0 \n
    - numpy-base=1.20.3=py39h74d4b33_0 \n
    - pip=21.2.4=pyhd8ed1ab_0 \n
    - dgl-cuda11.1=0.8.2=py39_0 \n
    - psutil=5.9.0=py39hb9d737c_1 \n
    - python=3.9.6=h49503c6_1_cpython \n
    - pytorch=1.9.0=py3.9_cuda11.1_cudnn8.0.5_0 \n
    - pytorch-cluster=1.5.9=py39_torch_1.9.0_cu111 \n
    - pytorch-geometric=1.7.2=py39_torch_1.9.0_cu111 \n
    - pytorch-scatter=2.0.8=py39_torch_1.9.0_cu111 \n
    - pytorch-sparse=0.6.11=py39_torch_1.9.0_cu111 \n
    - pytorch-spline-conv=1.2.1=py39_torch_1.9.0_cu111 \n
    - requests=2.26.0=pyhd8ed1ab_0 \n
    - requests-unixsocket=0.2.0=py_0 \n
    - scikit-learn=0.24.2=py39ha9443f7_0 \n
    - scipy=1.6.2=py39had2a1c9_1 \n
    - pip: \n
      - icecream==2.1.1 \n
      - lie-learn==0.0.1.post1 \n
      - opt-einsum==3.3.0 \n
      - e3nn==0.3.4 \n
  prefix: ./SE3-nvidia \n
  """
  )

In [4]:
#@title Install dependencies
#@markdown This may take 10-15 min but only needs to be done once per session. 
#@markdown The notebook will skip over this part if run again. 
%%bash -s 

set -e

if [ ! -f RFDES_READY ]; then
  echo "Cloning RFDesign..."
  git clone https://github.com/RosettaCommons/RFDesign.git
  touch RFDES_READY
fi

# setup conda
if [ ! -f CONDA_READY ]; then
  echo "Installing Conda..."
  wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
  bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
  rm Miniconda3-latest-Linux-x86_64.sh
  touch CONDA_READY
fi

if [ ! -f ENV_READY ]; then
  echo "Creating Conda SE3-nvidia environment for RFDesign"
  conda env create -f /content/env.yml 2>&1 1>/dev/null
  touch ENV_READY
fi

if [ ! -f BFF_WEIGHTS ]; then
  echo "Downloading BFF_mix_epoch25 weights..."
  wget http://files.ipd.uw.edu/pub/rfdesign/weights/BFF_mix_epoch25.pt >> /dev/null 2>&1  
  mkdir /content/RFDesign/inpainting/weights/
  mv /content/BFF_mix_epoch25.pt /content/RFDesign/inpainting/weights/BFF_mix_epoch25.pt
  touch BFF_WEIGHTS
fi


Cloning RFDesign...
Installing Conda...
Creating Conda SE3-nvidia environment for RFDesign


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda




Cloning into 'RFDesign'...
Checking out files: 100% (7342/7342), done.


In [5]:
#@title Run inpainting tests
#@markdown Test to see if installation was successful. Should end with "Successfully wrote output".
%%bash -s

if [ ! -f TEST_READY ]; then
  echo "Running inpainting tests..."
  source activate SE3-nvidia
  cd /content/RFDesign/inpainting/tests/
  conda init bash >> /dev/null 2>&1
  conda activate SE3-nvidia
  ./run_tests.sh
  touch TEST_READY
fi

Running inpainting tests...
Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Loading model checkpoint...
Successfully loaded model checkpoint
args before design
On design  0
asdfasf ['A1-20,5-5,A26-79']
on cycle 0
on cycle 1
on cycle 2
on cycle 3
on cycle 4
on cycle 5
on cycle 6
on cycle 7
on cycle 8
on cycle 9
on cycle 10
on cycle 11
on cycle 12
on cycle 13
On design  1
asdfasf ['A1-20,5-5,A26-79']
on cycle 0
on cycle 1
on cycle 2
on cycle 3
on cycle 4
on cycle 5
on cycle 6
on cycle 7
on cycle 8
on cycle 9
on cycle 10
on cycle 11
on cycle 12
on cycle 13


DGL backend not selected or invalid.  Assuming PyTorch for now.
ic| args: Namespace(pdb='2KL8.pdb', contigs=['A1-20,5,A26-79'], checkpoint='/content/RFDesign/inpainting/weights/BFF_mix_epoch25.pt', receptor_chain=None, inpaint_str=None, inpaint_seq=None, n_cycle=15, tmpl_conf='0.5', num_designs=2, res_translate=None, tie_translate=None, floating_points=None, block_rotate=None, verbose=False, out='out/2KL8_test', dump_pdb=True, dump_trb=True, dump_npz=True, dump_fasta=True, dump_all=True, input_json=None, outdir='out', prefix='2KL8_test')
ic| args.contigs: ['A1-20,5,A26-79']
ic| len(scpu): 79
ic| args.contigs: ['A1-20,5,A26-79']
ic| len(scpu): 79
Successfully wrote output


In [6]:
#@title Run local loop inpainting
%%bash -s $local_loop_path

LOCAL_LOOP_PATH=$1

source activate SE3-nvidia 
conda init bash >> /dev/null 2>&1
conda activate SE3-nvidia

for file in $LOCAL_LOOP_PATH*.sh; do
    chmod +x $file
    $file < /dev/null
done < /dev/null


Loading model checkpoint...
Successfully loaded model checkpoint
args before design
On design  0
asdfasf ['A1-27,2-2,A33-60']
on cycle 0
on cycle 1
on cycle 2
on cycle 3
on cycle 4
on cycle 5
on cycle 6
on cycle 7
on cycle 8
on cycle 9
on cycle 10
on cycle 11
on cycle 12
on cycle 13
on cycle 14
on cycle 15
on cycle 16
on cycle 17
on cycle 18
Loading model checkpoint...
Successfully loaded model checkpoint
args before design
On design  0
asdfasf ['A1-27,3-3,A33-60']
on cycle 0
on cycle 1
on cycle 2
on cycle 3
on cycle 4
on cycle 5
on cycle 6
on cycle 7
on cycle 8
on cycle 9
on cycle 10
on cycle 11
on cycle 12
on cycle 13
on cycle 14
on cycle 15
on cycle 16
on cycle 17
on cycle 18
Loading model checkpoint...
Successfully loaded model checkpoint
args before design
On design  0
asdfasf ['A1-27,4-4,A33-60']
on cycle 0
on cycle 1
on cycle 2
on cycle 3
on cycle 4
on cycle 5
on cycle 6
on cycle 7
on cycle 8
on cycle 9
on cycle 10
on cycle 11
on cycle 12
on cycle 13
on cycle 14
on cycle 15
on c

ic| args: Namespace(pdb='test_a94a8_template/able.pdb', contigs=['A1-27,2,A33-60'], checkpoint='/content/RFDesign/inpainting/weights/BFF_mix_epoch25.pt', receptor_chain=None, inpaint_str=None, inpaint_seq=None, n_cycle=20, tmpl_conf='0.5', num_designs=1, res_translate=None, tie_translate=None, floating_points=None, block_rotate=None, verbose=False, out='test_a94a8_template/local_loops/able_A1-27_2_A33-60', dump_pdb=True, dump_trb=True, dump_npz=True, dump_fasta=True, dump_all=True, input_json=None, outdir='test_a94a8_template/local_loops', prefix='able_A1-27_2_A33-60')
ic| args.contigs: ['A1-27,2,A33-60']
ic| len(scpu): 57
Successfully wrote output
ic| args: Namespace(pdb='test_a94a8_template/able.pdb', contigs=['A1-27,3,A33-60'], checkpoint='/content/RFDesign/inpainting/weights/BFF_mix_epoch25.pt', receptor_chain=None, inpaint_str=None, inpaint_seq=None, n_cycle=20, tmpl_conf='0.5', num_designs=1, res_translate=None, tie_translate=None, floating_points=None, block_rotate=None, verbose

In [7]:
#@title Get top 3 inpainted local loops by mean LDDT
import numpy as np
from collections import defaultdict
loop_dict = defaultdict(list)
for trb_file in os.listdir(local_loop_path):
  if trb_file.split('.')[-1] != 'trb':
    continue
  loop_info = trb_file.split('.')[0].split('_')
  loop_ends = (loop_info[1], loop_info[3])
  loop_size = loop_info[2]
  trb = np.load(local_loop_path + trb_file, allow_pickle=True)
  mean_lddt = np.array(trb['inpaint_lddt']).mean()
  loop_dict[loop_ends].append((mean_lddt, loop_size))

sorted_loop_dict = defaultdict(set)
for loop_ends, mean_lddts in loop_dict.items():
  sorted_loops = sorted(mean_lddts, reverse=True)[:3]
  for loop in sorted_loops:
    sorted_loop_dict[loop_ends].add(loop[-1])


In [8]:
#@title Make global loop-finding job files
def make_job_file(contigs, loop_sizes, outdir):
  if outdir[-1] != '/':
    outdir += '/'
  len_contigs = len(contigs)

  contig_str = ''
  for i in range(len_contigs - 1):
    contig_str += contigs[i]
    contig_str += ','
    contig_str += loop_sizes[i]
    contig_str += ','
  contig_str += contigs[-1]
  outname_suffix = '_'.join(contig_str.split(','))
  outname = outdir + pdbname + '_' + outname_suffix

  with open(f'{outname}.sh', 'w') as jobfile:
    jobfile.write('#!/bin/bash \n')
    pdbpath = custom_template_path + pdbname
    jobfile.write(f"pdb='{pdbpath}.pdb' \n")
    jobfile.write(f"contigs='{contig_str}' \n")
    jobfile.write(f"out='{outname}' \n")
    jobfile.write('num_designs=1 \n')
    jobfile.write('tmpl_conf=0.75 \n')
    script = "script='/content/RFDesign/inpainting/inpaint.py' \n"
    jobfile.write(script)
    jobfile.write('mkdir -p `dirname $out` \n')
    jobfile.write('python $script --pdb $pdb --contigs $contigs --dump_trb --dump_pdb --n_cycle 20 --out $out --num_designs $num_designs \n')

global_loop_path = custom_template_path + 'global_loops/'
if not os.path.exists(global_loop_path):
  os.mkdir(global_loop_path)

from itertools import product
sorted_contigs = sorted(sorted_loop_dict.keys())
sorted_contigs_ = sorted({v for k in sorted_contigs for v in k})
contigs_tuples = []
for sc in sorted_contigs_:
  chain = sc[0]
  first_resnum = int(sc[1:].split('-')[0])
  contigs_tuples.append((chain, first_resnum))
sorted_contigs_indices = sorted(range(len(sorted_contigs_)), key=lambda x: contigs_tuples[x])
sorted_contigs_ = [sorted_contigs_[i] for i in sorted_contigs_indices]

sorted_loop_pairs = list(product(*[sorted_loop_dict[sorted_contig] for sorted_contig in sorted_contigs]))
for sorted_loop_pair in sorted_loop_pairs:
  make_job_file(sorted_contigs_, sorted_loop_pair, global_loop_path)

In [9]:
#@title Run global loop inpainting
%%bash -s $global_loop_path

GLOBAL_LOOP_PATH=$1

source activate SE3-nvidia 
conda init bash >> /dev/null 2>&1
conda activate SE3-nvidia

for file in $GLOBAL_LOOP_PATH*.sh; do
    chmod +x $file
    $file 
done 


Loading model checkpoint...
Successfully loaded model checkpoint
args before design
On design  0
asdfasf ['A1-27,3-3,A33-60,3-3,A67-93,6-6,A101-126']
on cycle 0
on cycle 1
on cycle 2
on cycle 3
on cycle 4
on cycle 5
on cycle 6
on cycle 7
on cycle 8
on cycle 9
on cycle 10
on cycle 11
on cycle 12
on cycle 13
on cycle 14
on cycle 15
on cycle 16
on cycle 17
on cycle 18
Loading model checkpoint...
Successfully loaded model checkpoint
args before design
On design  0
asdfasf ['A1-27,3-3,A33-60,3-3,A67-93,7-7,A101-126']
on cycle 0
on cycle 1
on cycle 2
on cycle 3
on cycle 4
on cycle 5
on cycle 6
on cycle 7
on cycle 8
on cycle 9
on cycle 10
on cycle 11
on cycle 12
on cycle 13
on cycle 14
on cycle 15
on cycle 16
on cycle 17
on cycle 18
Loading model checkpoint...
Successfully loaded model checkpoint
args before design
On design  0
asdfasf ['A1-27,3-3,A33-60,3-3,A67-93,8-8,A101-126']
on cycle 0
on cycle 1
on cycle 2
on cycle 3
on cycle 4
on cycle 5
on cycle 6
on cycle 7
on cycle 8
on cycle 9
on c

ic| args: Namespace(pdb='test_a94a8_template/able.pdb', contigs=['A1-27,3,A33-60,3,A67-93,6,A101-126'], checkpoint='/content/RFDesign/inpainting/weights/BFF_mix_epoch25.pt', receptor_chain=None, inpaint_str=None, inpaint_seq=None, n_cycle=20, tmpl_conf='0.5', num_designs=1, res_translate=None, tie_translate=None, floating_points=None, block_rotate=None, verbose=False, out='test_a94a8_template/global_loops/able_A1-27_3_A33-60_3_A67-93_6_A101-126', dump_pdb=True, dump_trb=True, dump_npz=True, dump_fasta=True, dump_all=True, input_json=None, outdir='test_a94a8_template/global_loops', prefix='able_A1-27_3_A33-60_3_A67-93_6_A101-126')
ic| args.contigs: ['A1-27,3,A33-60,3,A67-93,6,A101-126']
ic| len(scpu): 120
Successfully wrote output
ic| args: Namespace(pdb='test_a94a8_template/able.pdb', contigs=['A1-27,3,A33-60,3,A67-93,7,A101-126'], checkpoint='/content/RFDesign/inpainting/weights/BFF_mix_epoch25.pt', receptor_chain=None, inpaint_str=None, inpaint_seq=None, n_cycle=20, tmpl_conf='0.5', 

In [10]:
#@title Get top 3 inpainted global loops by mean LDDT
mean_lddts = []
for trb_file in os.listdir(global_loop_path):
  if trb_file.split('.')[-1] != 'trb':
    continue
  trb = np.load(global_loop_path + trb_file, allow_pickle=True)
  mean_lddt = np.array(trb['inpaint_lddt']).mean()
  mean_lddts.append((mean_lddt, trb_file))

sorted_loops = sorted(mean_lddts, reverse=True)[:3]

output_loop_path = custom_template_path + 'output/'
if not os.path.exists(output_loop_path):
  os.mkdir(output_loop_path)

import shutil
for i, loopfile in enumerate(sorted_loops):
  pdbname_ = loopfile[-1].split('.')[0]
  pdb = pdbname_ + '.pdb'
  pdbpath = global_loop_path + pdb
  dst = output_loop_path + pdbname_ + '_rank_' + str(i + 1) + '_' + str(np.round(loopfile[0], 4)) + '.pdb'
  shutil.copyfile(pdbpath, dst)


In [11]:
#@title Package and download results
#@markdown If you are having issues downloading the result archive, try disabling your adblocker and run this cell again. If that fails click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).

!zip -FSr $jobname".result.zip" $custom_template_path"output"
files.download(f"{jobname}.result.zip")


  adding: test_a94a8_template/output/ (stored 0%)
  adding: test_a94a8_template/output/able_A1-27_5_A33-60_6_A67-93_7_A101-126_0_rank_1_0.9601.pdb (deflated 75%)
  adding: test_a94a8_template/output/able_A1-27_5_A33-60_3_A67-93_7_A101-126_0_rank_3_0.9565.pdb (deflated 75%)
  adding: test_a94a8_template/output/able_A1-27_5_A33-60_9_A67-93_7_A101-126_0_rank_2_0.9569.pdb (deflated 75%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Instructions <a name="Instructions"></a>
**Quick start**
1. Paste your protein sequence(s) in the input field.
2. Press "Runtime" -> "Run all".

**Result zip file contents**

1. PDB formatted structures sorted by avg. pLDDT of loops. File names include the rank (lower is better) and avg. pLDDT (higher is better), averaged over all inpainted loops. 

At the end of the job a download modal box will pop up with a `jobname.result.zip` file. Note that this protocol is not extensively tested. It might break for your protein. Use at own risk!

